# a. Khởi động Spark Session:

In [4]:
from pyspark.sql import SparkSession

# Tạo SparkSession
spark = SparkSession.builder \
    .appName("University Data Lake Analysis") \
    .getOrCreate()




# b. Đọc dữ liệu từ HDFS vào DataFrames:

In [5]:
students = spark.read.csv("hdfs://localhost:9000/data/students_expanded.csv", header=True, inferSchema=True)
courses = spark.read.csv("hdfs://localhost:9000/data/courses_expanded.csv", header=True, inferSchema=True)
rooms = spark.read.csv("hdfs://localhost:9000/data/rooms_expanded.csv", header=True, inferSchema=True)
devices = spark.read.csv("hdfs://localhost:9000/data/devices_expanded.csv", header=True, inferSchema=True)
logs = spark.read.csv("hdfs://localhost:9000/data/logs_expanded.csv", header=True, inferSchema=True)

# Hiển thị schema và một số dữ liệu
students.printSchema()
students.show(5)
courses.show(5)

root
 |-- Student_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Major: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)

+----------+--------------+----------------+----+--------+
|Student_ID|          Name|           Major|Year|  Status|
+----------+--------------+----------------+----+--------+
|      2001|Nguyễn Văn Anh|Khoa học dữ liệu|   1|Inactive|
|      2002|  Lê Thị Hương|Khoa học dữ liệu|   2|Inactive|
|      2003|Trần Văn Khánh|Khoa học dữ liệu|   3|  Active|
|      2004|  Phạm Thị Yến|Khoa học dữ liệu|   4|  Active|
|      2005|Hoàng Minh Trí|Khoa học dữ liệu|   1|  Active|
+----------+--------------+----------------+----+--------+
only showing top 5 rows

+---------+--------------------+-------+--------------------+
|Course_ID|         Course_Name|Credits|          Instructor|
+---------+--------------------+-------+--------------------+
|    DS101|Nhập môn Khoa học...|      3|  TS. Nguyễn Thị Hoa|

# c. Phân tích dữ liệu:

1. Phân tích tỷ lệ sinh viên theo trạng thái học tập:

In [6]:
students.groupBy("Status").count().show()


+--------+-----+
|  Status|count|
+--------+-----+
|  Active|    8|
|Inactive|    2|
+--------+-----+



2. Xác định lớp học đông/ít sinh viên (mô phỏng):

In [7]:
students.groupBy("Major").count().orderBy("count", ascending=False).show()


+----------------+-----+
|           Major|count|
+----------------+-----+
|Khoa học dữ liệu|   10|
+----------------+-----+



3.  Phân tích lịch sử sử dụng phòng học:

In [8]:
logs.groupBy("Room_ID", "Event").count().show()



+-------+------------------+-----+
|Room_ID|             Event|count|
+-------+------------------+-----+
|   C104|Device Maintenance|    1|
|   C102|   Class Scheduled|    1|
|   C203|   Class Scheduled|    1|
|   C205|Device Maintenance|    1|
|   C203|Device Maintenance|    1|
|   C205|   Class Scheduled|    1|
|   C102|Device Maintenance|    1|
|   C201|   Class Scheduled|    1|
|   C201|Device Maintenance|    1|
|   C204|     Room Cleaning|    1|
+-------+------------------+-----+



# d. Lưu kết quả vào HDFS:

In [9]:
result = students.groupBy("Status").count()
result.write.mode("overwrite").parquet("hdfs://localhost:9000/data/output/student_status")

# Kiểm tra file kết quả trên HDFS
!hdfs dfs -ls /data/output/student_status


Found 2 items
-rw-r--r--   3 vantai supergroup          0 2024-12-19 23:35 /data/output/student_status/_SUCCESS
-rw-r--r--   3 vantai supergroup        743 2024-12-19 23:35 /data/output/student_status/part-00000-5915513f-2049-4b18-af7e-b12bc62e0a41-c000.snappy.parquet


In [10]:
students.describe().show()

24/12/19 23:35:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+--------------+----------------+------------------+--------+
|summary|        Student_ID|          Name|           Major|              Year|  Status|
+-------+------------------+--------------+----------------+------------------+--------+
|  count|                10|            10|              10|                10|      10|
|   mean|            2005.5|          null|            null|               2.3|    null|
| stddev|3.0276503540974917|          null|            null|1.1595018087284057|    null|
|    min|              2001|  Bùi Thị Thảo|Khoa học dữ liệu|                 1|  Active|
|    max|              2010|Đặng Quang Huy|Khoa học dữ liệu|                 4|Inactive|
+-------+------------------+--------------+----------------+------------------+--------+

